In [215]:
#https://www.freecodecamp.org/news/deploy-ml-model-to-production-as-api/

In [216]:
import pandas as pd
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer(language='english')
from nltk.stem.wordnet import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split,cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
import pickle
import numpy as np
import matplotlib.pyplot as plt
from string import punctuation 
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
from cleantext import clean
from wordcloud import WordCloud, STOPWORDS

# Download dependency
for dependency in (
    "brown",
    "names",
    "wordnet",
    "averaged_perceptron_tagger",
    "universal_tagset",
    "stopwords"
):
    nltk.download(dependency)

#nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")
# seeding
np.random.seed(123)

[nltk_data] Downloading package stopwords to /Users/ckn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ckn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /Users/ckn/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package names to /Users/ckn/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ckn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ckn/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/ckn/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ckn/n

In [217]:
reviews=pd.read_csv('Fire Stick.csv')
reviews.dropna(axis=0,subset=['text'],inplace=True)
reviews=reviews.drop_duplicates(subset=['id'],keep='first')

In [218]:
reviews

,id,profileName,text,date,title,rating,images,helpful,Configuration
0,R3EUSNDMBJK2X2,kushal kumar,I am useing Sony bx32 modal tv(2012) it have 2...,Reviewed in India on 9 May 2021,NOT supporting to SONY TV'S,1,https://images-na.ssl-images-amazon.com/images...,290,Fire TV Stick
1,R1WACJL2O37SWG,Ishpreet Singh,Everything is great but i bought it for 3999/-...,Reviewed in India on 17 June 2021,Working good but disappointed with price drop,2,https://images-na.ssl-images-amazon.com/images...,276,Fire TV Stick
2,R21CJ1BDI98AH9,Sudharsan A,Product works well upto 1year... Then u need t...,Reviewed in India on 21 June 2021,Worst remote ever... Replacement of remote is ...,1,NaN,129,Fire TV Stick
3,R3Q6IHW1DEAMK,Priyank Agrawal,There is no customer support or details availa...,Reviewed in India on 20 June 2021,Not compatible with my Sony LED TV,1,NaN,108,Fire TV Stick
4,R3DARG068E5AKX,Suryatej,In the product description it says 'Made in In...,Reviewed in India on 24 July 2021,Made in China,3,https://m.media-amazon.com/images/I/B1+yXJ+1uo...,71,Fire TV Stick
...,...,...,...,...,...,...,...,...,...
4995,R29G0GBL9946I1,Srinivasa Rao V,Ok to use . Led can be used as smart tv. But v...,Reviewed in India on 4 October 2021,Ok,4,NaN,0,Fire TV Stick
4996,R25BE45ZD1FGWZ,Kishan jasani,Best streamGood content alsoI personally like ...,Reviewed in India on 9 August 2021,Very n best product,5,NaN,0,Fire TV Stick
4997,ROL7Y2H1WP5LB,Nagireddy,I gifted to my relatives and they liked it soo...,Reviewed in India on 22 November 2021,TV become smart with this device,5,NaN,0,Fire TV Stick
4998,R263CYE5T7SW6N,sumant sharma,Awesome as usual. This is my third Fire stick ...,Reviewed in India on 18 October 2021,Awesome way to convert a lifeless TV into a sm...,5,NaN,0,Fire TV Stick


In [219]:
def label(value):
    if value>3:
        return 1
    elif value<3:
        return -1
    else:
        return 0

In [220]:
reviews['label']=reviews['rating'].apply(label)
reviews.drop(columns=['id','profileName','date','title','images','Configuration','helpful'],axis=1,inplace=True)

In [221]:
reviews

,text,rating,label
0,I am useing Sony bx32 modal tv(2012) it have 2...,1,-1
1,Everything is great but i bought it for 3999/-...,2,-1
2,Product works well upto 1year... Then u need t...,1,-1
3,There is no customer support or details availa...,1,-1
4,In the product description it says 'Made in In...,3,0
...,...,...,...
4995,Ok to use . Led can be used as smart tv. But v...,4,1
4996,Best streamGood content alsoI personally like ...,5,1
4997,I gifted to my relatives and they liked it soo...,5,1
4998,Awesome as usual. This is my third Fire stick ...,5,1


In [222]:
def text_preprocessor(comment):
    #removing http links
    comment=re.sub(r"http\S+"," ",comment)
    #removing special tags
    comment=BeautifulSoup(comment,'lxml').get_text()
    #decontracing the words
    comment = re.sub(r"won't", "will not", comment)
    comment = re.sub(r"can\'t", "can not",comment)
    # general
    comment = re.sub(r"n\'t", " not", comment)
    comment = re.sub(r"\'re", " are", comment)
    comment= re.sub(r"\'s", " is",comment)
    comment= re.sub(r"\'d", " would", comment)
    comment= re.sub(r"\'ll", " will", comment)
    comment= re.sub(r"\'t", " not", comment)
    comment= re.sub(r"\'ve", " have", comment)
    comment= re.sub(r"\'m", " am",comment)
    #removing words with numbers
    comment=re.sub("\S*\d\S*",' ',comment)
    #removing special characters
    comment=re.sub("[^A-Za-z0-9]+",' ',comment)
    #removing numbers
    comment=re.sub("\d+",' ',comment)
    #removing stopwords & stemming
    return ' '.join(stemmer.stem(word.lower()) for word in comment.split() if word not in stopwords.words('english'))

In [223]:
reviews["clean_message"] = reviews["text"].apply(text_preprocessor)

In [224]:
X_train, X_test, y_train, y_test = train_test_split(
    reviews["clean_message"],
    reviews["label"],
    test_size=0.25,
    random_state=43,
    shuffle=True,
    stratify=reviews["label"],
)

In [225]:
# Transform text data 
vectorizer = CountVectorizer(lowercase=False)
vectorizer.fit(X_train)

#transform train data 
X_train_trans = vectorizer.transform(X_train)

#transform test data
X_text_trans = vectorizer.transform(X_test)

In [226]:
spam_classifier = MultinomialNB()

In [227]:
# Train the model with cross validation
scores = cross_val_score(spam_classifier,X_train_trans,y_train,cv=10,verbose=3,n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[CV] END ................................ score: (test=0.847) total time=   0.0s
[CV] END ................................ score: (test=0.828) total time=   0.0s
[CV] END ................................ score: (test=0.837) total time=   0.0s
[CV] END ................................ score: (test=0.839) total time=   0.0s
[CV] END ................................ score: (test=0.817) total time=   0.0s
[CV] END ................................ score: (test=0.826) total time=   0.0s
[CV] END ................................ score: (test=0.826) total time=   0.0s
[CV] END ................................ score: (test=0.806) total time=   0.0s
[CV] END ................................ score: (test=0.814) total time=   0.0s
[CV] END ................................ score: (test=0.872) total time=   0.0s


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


In [228]:
scores.mean()

0.8311966766426945

In [229]:
# fine turning model parameters

distribution = {"alpha": [1, 0.1, 0.01, 0.001, 0.0001, 0, 0.2, 0.3]}

grid = RandomizedSearchCV(
    spam_classifier,
    param_distributions=distribution,
    n_jobs=-1,
    cv=10,
    n_iter=20,
    random_state=42,
    return_train_score=True,
    verbose=2,
)

In [230]:
# training with randomized search
grid.fit(X_train_trans, y_train)

Fitting 10 folds for each of 8 candidates, totalling 80 fits
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END ..........................................alpha=0.1; total time=   0.0s
[CV] END .......................................

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/naive_bayes.py:890: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/naive_bayes.py:890: RuntimeWarning: divide by zero e

[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END ............................................alpha=1; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s
[CV] END .........................................alpha=0.01; total time=   0.0s


RandomizedSearchCV(cv=10, estimator=MultinomialNB(), n_iter=20, n_jobs=-1,
                   param_distributions={'alpha': [1, 0.1, 0.01, 0.001, 0.0001,
                                                  0, 0.2, 0.3]},
                   random_state=42, return_train_score=True, verbose=2)

In [231]:
spam_classifier.fit(X_train_trans, y_train)

MultinomialNB()

In [232]:
# summarize the results of the random parameter search
print(grid.best_score_)
print(grid.best_estimator_)
print(grid.best_params_)

0.8311966766426945
MultinomialNB(alpha=1)
{'alpha': 1}


In [233]:
# predict on the test data
y_pred = spam_classifier.predict(X_text_trans)


In [234]:
# check accuracy score
accuracy_score(y_test, y_pred)

0.8331970564186427

In [235]:
#save model 
import joblib 

joblib.dump(spam_classifier, 'emotion-detection-model.pkl')

['emotion-detection-model.pkl']

['count_vectorizer.pkl']

In [237]:
pipeline= Pipeline([
('bow', CountVectorizer(analyzer=text_preprocessor)),  # strings to token integer counts
('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
('classifier', RandomForestClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [238]:
pipeline.fit(X_train,y_train)
y_preds = pipeline.predict(X_test)
accuracy_score(y_test,y_preds)

0.7563368765331153

In [239]:
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [240]:
nb_m= MultinomialNB(alpha= 1)
dt=DecisionTreeClassifier()
et=ExtraTreeClassifier()
lt=LogisticRegression()
rf=RandomForestClassifier()
svc=SVC()
models = [nb_m,dt,et,lt,rf,svc]

In [241]:
def check(model):
    pipeline= Pipeline([
    ('bow', CountVectorizer(analyzer=text_preprocessor)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', model),  # train on TF-IDF vectors w/ Naive Bayes classifier
    ])
    pipeline.fit(X_train,y_train)
    y_preds = pipeline.predict(X_test)
    return accuracy_score(y_test,y_preds)
    
    

In [242]:
acc_scores = []
for model  in models:
    res =check(model)
    print(model,res)

MultinomialNB(alpha=1) 0.7301717089125103
DecisionTreeClassifier() 0.6688470973017171
ExtraTreeClassifier() 0.6508585445625511
LogisticRegression() 0.7432542927228127
RandomForestClassifier() 0.7555192150449714
SVC() 0.758789860997547


In [243]:
bow=CountVectorizer(analyzer=text_preprocessor,ngram_range=(2,3)).fit(reviews['text'])
bow_transformer=bow.transform(reviews['text'])
transformer=TfidfTransformer().fit(bow_transformer)
tfidf=transformer.transform(bow_transformer)
X=tfidf
y=reviews['label']
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25)

In [257]:
classifier = RandomForestClassifier().fit(X_train,y_train)
pred_y_train = classifier.predict(X_train)
print(accuracy_score(y_train,pred_y_train))

0.9923643305154077


In [258]:
pred_y_test = classifier.predict(X_test)
print(accuracy_score(y_test,pred_y_test))

0.7522485690923958


In [ ]:
#save Vectorizer
joblib.dump(bow,'count_vectorizer.pkl')

In [260]:
#save model 
import joblib 
joblib.dump(classifier, 'model.pkl')

['model.pkl']